In [1]:
import pandas as pd
import sys
sys.path.insert(0, "../..")
from skpref.random_utility import BradleyTerry
from skpref.task import ChoiceTask
from sklearn.model_selection import train_test_split
import pickle
from collections import Counter
from skpref.model_selection import GridSearchCV

In [2]:
with open('../examples/data/product_choices.pickle', 'rb') as handle:
    choice_data = pickle.load(handle)
    
with open('../examples/data/product_info.pickle', 'rb') as handle:
    product_data = pickle.load(handle)

In [3]:
choice_data.head()

,STORE_ID,BASKET_ID,choice,alternatives
0,286,29483925362,[6396581],"[5582789, 5584007, 951703, 6396581]"
1,286,29773175255,[5584007],"[5582789, 5584007, 951703, 6396581]"
2,286,30673440294,[5584007],"[5582789, 5584007, 951703, 6396581]"
3,286,32269485930,[951703],"[5582789, 5584007, 951703, 6396581]"
4,286,40876776136,[5582789],"[5582789, 5584007, 951703, 6396581]"


In [4]:
product_data.head()

,PRODUCT_ID,prod_size,price,price_per_size
0,849098,7.0,1.021950,0.145993
1,951703,4.0,0.568998,0.142249
2,5578643,6.0,0.393425,0.065571
3,5582789,6.0,0.392923,0.065487
4,5584007,6.0,0.393564,0.065594


In [5]:
train, test = train_test_split(choice_data, random_state=1, test_size=0.1)

In [6]:
products_bought_train = ChoiceTask(train, 'alternatives', 'choice', features_to_use=['price_per_size', 'prod_size'], secondary_table=product_data,
                             secondary_to_primary_link={"PRODUCT_ID": ['alternatives', 'choice']})
products_bought_test = ChoiceTask(test, 'alternatives', 'choice', features_to_use=['price_per_size', 'prod_size'], secondary_table=product_data,
                             secondary_to_primary_link={"PRODUCT_ID": ['alternatives', 'choice']})

In [7]:
mybt = BradleyTerry(method='BFGS', alpha=1e-5)
mybt.fit_task(products_bought_train)

In [8]:
# find most popular product ids
counter = Counter()
j = []
for i in choice_data.choice.values:
    j = j + [k for k in i]
for z in j:
    counter[z] += 1

print('Most popular product_ids based on amounts purchased')
print(counter.most_common())
print("Bradley Terry rank of items given the covariates")
print(mybt.rank_entities(ascending=False))

Most popular product_ids based on amounts purchased
[(5584007, 626), (5582789, 503), (5585727, 439), (5586076, 331), (951703, 266), (5584100, 248), (849098, 219), (5578643, 210), (6396581, 156), (5591746, 2)]
Bradley Terry rank of items given the covariates
[5584007 5582789  951703 5585727 5586076  849098 5578643 5584100 6396581
 5591746]


In [9]:
mybt.predict_proba_task(products_bought_test)

array([0.58825521, 0.6658351 , 0.73734034, ..., 0.49196996, 0.62266418,
       0.40702812])

In [10]:
mybt.predict_choice_task(products_bought_test)

array([5584007, 5584007, 5584007, ...,  951703,  849098, 5585727],
      dtype=int64)

In [11]:
mybt.predict_task(products_bought_test)

array([1, 1, 1, ..., 0, 1, 0])

# Example using Grid Search

In [12]:
products_bought_train = ChoiceTask(train, 'alternatives', 'choice', features_to_use=None)
products_bought_test = ChoiceTask(test, 'alternatives', 'choice', features_to_use=None)
to_tune = {'alpha': [0.1, 0.5, 1], 'method': ['BFGS', 'Newton-CG']}
gs_bt = GridSearchCV(BradleyTerry(), to_tune,  cv=3)
gs_bt.fit_task(products_bought_train)
gs_bt.inspect_results()

The model with the best parameters was:
BradleyTerry(alpha=0.5, initial_params=None, max_iter=None, method='BFGS',
       tol=1e-05)
With a score of -0.645979957349246
All the trials results summarised in descending score
   alpha     method  mean_test_score
2    0.5       BFGS        -0.645980
3    0.5  Newton-CG        -0.645980
4    1.0       BFGS        -0.646007
5    1.0  Newton-CG        -0.646007
0    0.1       BFGS        -0.646209
1    0.1  Newton-CG        -0.646209


In [13]:
gs_bt.rank_entities(ascending=False)

[5584007,
 5582789,
 5585727,
 5586076,
 951703,
 849098,
 5578643,
 5584100,
 6396581,
 5591746]

In [14]:
gs_bt.predict_proba_task(products_bought_test)

array([0.41180851, 0.66568507, 0.73717355, ..., 0.50803425, 0.62245114,
       0.40701829])